# GNN Hyperparameter Optimization with WandB Sweeps

This notebook demonstrates how to use WandB sweeps for hyperparameter optimization of GNN models.

In [1]:
# Import necessary libraries
import torch
import numpy as np
import pandas as pd
import ast
import wandb
from types import SimpleNamespace
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Import custom modules
from GraphBuilder_with_features import create_graph_dataset
from sweep_utils import (
    run_sweep, 
    quick_sweep,
    analyze_sweep_results,
    create_example_config_file
)

In [7]:
# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

## 1. Load and Prepare Data

In [8]:
# Load data function
def load_graph_data(loop):
    """Load graph data from CSV files."""
    edges = []
    y = []
    
    filename = f'../Graph_Edge_Data/den_graph_data_{loop}.csv'
    df = pd.read_csv(filename)
    edges += df['EDGES'].tolist()
    y += df['COEFFICIENTS'].tolist()
    
    edges = [ast.literal_eval(e) for e in edges]
    graphs_data = list(zip(edges, y))
    return graphs_data

In [9]:
# Load data
graphs_data = load_graph_data(loop=7)
print(f"Loaded {len(graphs_data)} graphs")

Loaded 164 graphs


In [10]:
# Create dataset with chosen features
feature_config = {
    'selected_features': ['basic', 'face', 'spectral_node', 'centrality'],
    'laplacian_pe_k': 3
}

dataset, scaler = create_graph_dataset(graphs_data, feature_config)
print(f"Dataset created with {len(dataset)} graphs")
print(f"Feature dimensions: {dataset[0].x.shape[1]}")
print(f"Feature names: {dataset[0].feature_names}")

Extracting features...
Normalizing features...
Created dataset with 164 graphs
Feature dimensions: 13
Feature names: ['degree', 'num_faces', 'avg_face_size', 'max_face_size', 'face_size_variance', 'fiedler_vector', 'eigenvector_energy', 'third_eigenvector', 'betweenness_centrality', 'closeness_centrality', 'eigenvector_centrality', 'clustering_coefficient', 'pagerank']
Dataset created with 164 graphs
Feature dimensions: 13
Feature names: ['degree', 'num_faces', 'avg_face_size', 'max_face_size', 'face_size_variance', 'fiedler_vector', 'eigenvector_energy', 'third_eigenvector', 'betweenness_centrality', 'closeness_centrality', 'eigenvector_centrality', 'clustering_coefficient', 'pagerank']


## 2. Define Hyperparameter Search Space

In [12]:
# Define hyperparameter ranges for grid search
param_ranges = {
    'hidden_channels': [32, 64],
    'num_layers': [2, 3, 4],
    'dropout': [0.1, 0.2, 0.3],
    'lr': [0.001, 0.003, 0.01],
    'weight_decay': [0, 1e-4, 5e-4]
}

# Calculate total number of combinations
total_runs = 1
for param, values in param_ranges.items():
    total_runs *= len(values)
    print(f"{param}: {len(values)} values - {values}")

print(f"\nTotal combinations: {total_runs}")

hidden_channels: 2 values - [32, 64]
num_layers: 3 values - [2, 3, 4]
dropout: 3 values - [0.1, 0.2, 0.3]
lr: 3 values - [0.001, 0.003, 0.01]
weight_decay: 3 values - [0, 0.0001, 0.0005]

Total combinations: 162


In [13]:
# Fixed configuration (not swept)
fixed_config = {
    'model_name': 'gin',
    'epochs': 100,
    'batch_size': 32,
    'scheduler_type': 'onecycle',
    'save_models': False
}

## 3. Run Hyperparameter Sweep

### Option A: Quick Test (Fewer Combinations)

In [14]:
# Quick test with fewer combinations
quick_param_ranges = {
    'hidden_channels': [32, 64],
    'num_layers': [2, 3],
    'dropout': [0.1, 0.2],
    'lr': [0.001, 0.01],
    'weight_decay': [0, 1e-4]
}

# Calculate combinations
quick_runs = 1
for values in quick_param_ranges.values():
    quick_runs *= len(values)
print(f"Quick test combinations: {quick_runs}")

Quick test combinations: 32


In [16]:
# Run quick sweep
project_name = "gnn-planar-graphs-sweep"
sweep_name = "quick_test"

# Uncomment to run:
sweep_id = run_sweep(
    param_ranges=quick_param_ranges,
    dataset=dataset,
    project_name=project_name,
    fixed_config=fixed_config,
    sweep_name=sweep_name
)

Create sweep with ID: dtjsh4vm
Sweep URL: https://wandb.ai/dian-gabriele-desydeutsches-elektronen-synchrotron/gnn-planar-graphs-sweep/sweeps/dtjsh4vm


wandb: Agent Starting Run: drflzuhk with config:
wandb: 	dropout: 0.1
wandb: 	hidden_channels: 32
wandb: 	lr: 0.001
wandb: 	num_layers: 2
wandb: 	weight_decay: 0
wandb: Currently logged in as: dian-gabriele (dian-gabriele-desydeutsches-elektronen-synchrotron) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Using device: cpu
Train size: 131, Val size: 33



Starting training...
Model architecture: gin
Hidden dim: 32, Layers: 2
Initial LR: 0.00011999999999999988
Epoch   0/100: Train Loss=0.9230, Acc=0.3893, Val Loss=0.7604, Acc=0.3333, LR=0.000128
Epoch  10/100: Train Loss=0.5652, Acc=0.6489, Val Loss=0.6640, Acc=0.7273, LR=0.000984
Epoch  20/100: Train Loss=0.3880, Acc=0.8397, Val Loss=0.6639, Acc=0.6364, LR=0.002424
Epoch  30/100: Train Loss=0.2022, Acc=0.9389, Val Loss=0.9727, Acc=0.6970, LR=0.002998
Epoch  40/100: Train Loss=0.2861, Acc=0.8626, Val Loss=0.8694, Acc=0.6667, LR=0.002814
Epoch  50/100: Train Loss=0.2642, Acc=0.9618, Val Loss=0.6418, Acc=0.7273, LR=0.002371
Epoch  60/100: Train Loss=0.1653, Acc=0.9389, Val Loss=1.0031, Acc=0.6667, LR=0.001755
Epoch  70/100: Train Loss=0.1403, Acc=0.9542, Val Loss=1.0522, Acc=0.6970, LR=0.001088
Epoch  80/100: Train Loss=0.1346, Acc=0.9466, Val Loss=1.3618, Acc=0.6061, LR=0.000503
Epoch  90/100: Train Loss=0.1331, Acc=0.9466, Val Loss=1.2771, Acc=0.6667, LR=0.000116
Epoch  99/100: Train Lo

current_lr,▂▂▂▂▂▄▄▅▆▆██████▇▇▇▇▇▆▆▆▆▅▅▅▅▅▄▄▃▃▂▂▂▁▁▁
epoch,▁▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▄▄▄▄▄▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_accuracy,▁▂▃▄▄▅▅▄▅▆▇▇▆▆▆▆▇▆▇█▇▇▇█▇▇▇▇█▇█████████▇
train_f1,▁▄▅▅▄▅▅▆▆▆▇▆▆▆▇▇▇▆▇█▇▇▇█▇▇▇▇███▇███████▇
train_loss,█▇▆▆▆▅▅▅▅▅▄▃▃▄▄▃▃▃▃▃▃▂▃▂▂▃▃▂▂▁▁▂▁▁▁▁▁▁▁▁
train_precision,▂▂▂▂▁▃▃▄▆▃▇▆▇▄▅▅▆██▇▆▇▇▇▇▆▆▇██▆▇▇█▇▇█▇▇▇
train_recall,▁▁▃▇█▇▇▇▆▇▆▇▆▇▇█▆▇▇▇█▆██▇████▇██████████
val_accuracy,▁▆▇▇▆▇█▇▇▇▆▆▆▆▆▇▅▇▆▅▆▆▆█▆▅▅▇▆▆▇▆█▇▆▇▇▇▇▇
val_f1,▄▇██▅▆▅▄▇▆▇▆▇▅█▁██▇▇▆▇▆▂▃▅▆▆▇▇██▇▇█▇▇▆▇▇
val_loss,▃▂▂▂▂▂▁▂▂▂▄▇▃▂▄▃▂▂▂▁▇▅▅▄▅▆▇█▆▆▇▇██▇▆▇▇▇▇
val_precision,▁▆▂▁▄▅▄▇▂▆▁▄▃▆▇▄▃▅▃▄█▃▅▅▆▇▅▆▅▅██▇▇▅▅▅▅▆▆


...

wandb: ERROR Run meam5c6v errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/home/gabriele/Documents/GitHub/ML-correlator/.venv/lib/python3.12/site-packages/wandb/agents/pyagent.py", line 306, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "/home/gabriele/Documents/GitHub/ML-correlator/Sweep_Claude_4/sweep_utils.py", line 175, in train_fn
wandb: ERROR     train_sweep_iteration(dataset, fixed_config,
wandb: ERROR   File "/home/gabriele/Documents/GitHub/ML-correlator/Sweep_Claude_4/sweep_utils.py", line 129, in train_sweep_iteration
wandb: ERROR     wandb.log({
wandb: ERROR   File "/home/gabriele/Documents/GitHub/ML-correlator/.venv/lib/python3.12/site-packages/wandb/sdk/lib/preinit.py", line 36, in preinit_wrapper
wandb: ERROR     raise wandb.Error(f"You must call wandb.init() before {name}()")
wandb: ERROR wandb.errors.errors.Error: You must call wandb.init() before wandb.log()
wandb: ERROR 
wandb: Agent Starting Run: v24n2d4t with config:

Using device: cpu
Train size: 131, Val size: 33



Starting training...
Model architecture: gin
Hidden dim: 32, Layers: 3
Initial LR: 0.00011999999999999988
Epoch   0/100: Train Loss=0.7642, Acc=0.5038, Val Loss=0.6603, Acc=0.5152, LR=0.000128
Epoch  10/100: Train Loss=0.5154, Acc=0.7176, Val Loss=0.6362, Acc=0.6364, LR=0.000984
Epoch  20/100: Train Loss=0.3296, Acc=0.8244, Val Loss=0.5706, Acc=0.7879, LR=0.002424
Epoch  30/100: Train Loss=0.2841, Acc=0.8779, Val Loss=1.3939, Acc=0.5152, LR=0.002998
Epoch  40/100: Train Loss=0.2145, Acc=0.9237, Val Loss=1.3759, Acc=0.6061, LR=0.002814
Epoch  50/100: Train Loss=0.1633, Acc=0.9389, Val Loss=1.3571, Acc=0.5152, LR=0.002371
Epoch  60/100: Train Loss=0.2136, Acc=0.9008, Val Loss=1.2677, Acc=0.5758, LR=0.001755
Epoch  70/100: Train Loss=0.0844, Acc=0.9695, Val Loss=1.6803, Acc=0.6061, LR=0.001088
Epoch  80/100: Train Loss=0.0726, Acc=0.9847, Val Loss=1.6988, Acc=0.6061, LR=0.000503
Epoch  90/100: Train Loss=0.0714, Acc=0.9924, Val Loss=1.7278, Acc=0.5758, LR=0.000116
Epoch  99/100: Train Lo

current_lr,▁▂▂▂▂▅▅▆▆▆▇▇█████▇▇▇▆▆▆▅▅▄▄▄▄▃▃▃▂▂▂▂▁▁▁▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇███
train_accuracy,▁▂▅▅▅▅▅▆▆▆▆▆▆▆▇▆▆▇▇▇▇▇▇▇▇█▇▇█▇██▇▇███▇██
train_f1,▁▂▅▅▆▅▅▆▇▆▆▇▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███▇▇▇█████
train_loss,█▇▇▆▅▄▃▃▅▄▄▄▄▂▃▃▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▂▂▁▁▁▁▁▁▂
train_precision,▁▁▁▄▃▃▄▆▆▄▆▅▇▇▆▆▆▇▆▅▇█▇▇█▇▆▇▇▇▇▇▇▇██▇▇██
train_recall,▁▅▅▃▃▂▃▇▇▅▅▇█▆▅▇▆▆█▆██▇▇█████▇▆▇████▆███
val_accuracy,▃▁▂▃▅▄▄▇▇▇▇▇█▆▅▇▅▇▆▄▆█▅▆▆▄▄▃▆▄▅▄▅▅▄▄▄▄▄▄
val_f1,▁▂▄▄▄▄▅██▄▇▅▅▇▇▅▇▆▆▄██▅▆▆▄▄▄▃▅▄▄▅▄▄▄▄▄▄▄
val_loss,▁▁▁▁▁▁▁▂▂▂▃▅▄▄▂▅▇▇▅▅▂▂▃▄▄▅▄▆▆▇█████▇▇███
val_precision,▃▁▄▄▆▆▅▆▇█▅▆▅▅▅▅▄▆▅▅█▅▅▅▄▆▅▅▅▅▅▅▅▅▄▅▅▅▅▅


wandb: ERROR Run v24n2d4t errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/home/gabriele/Documents/GitHub/ML-correlator/.venv/lib/python3.12/site-packages/wandb/agents/pyagent.py", line 306, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "/home/gabriele/Documents/GitHub/ML-correlator/Sweep_Claude_4/sweep_utils.py", line 175, in train_fn
wandb: ERROR     train_sweep_iteration(dataset, fixed_config,
wandb: ERROR   File "/home/gabriele/Documents/GitHub/ML-correlator/Sweep_Claude_4/sweep_utils.py", line 129, in train_sweep_iteration
wandb: ERROR     wandb.log({
wandb: ERROR   File "/home/gabriele/Documents/GitHub/ML-correlator/.venv/lib/python3.12/site-packages/wandb/sdk/lib/preinit.py", line 36, in preinit_wrapper
wandb: ERROR     raise wandb.Error(f"You must call wandb.init() before {name}()")
wandb: ERROR wandb.errors.errors.Error: You must call wandb.init() before wandb.log()
wandb: ERROR 
wandb: ERROR Detected 3 failed runs in the first

### Option B: Full Grid Search

In [ ]:
# Full sweep - WARNING: This will run many experiments!
# sweep_id = run_sweep(
#     param_ranges=param_ranges,
#     dataset=dataset,
#     project_name=project_name,
#     fixed_config=fixed_config,
#     sweep_name="full_grid_search"
# )

### Option C: Using the Quick Sweep Function

In [ ]:
# Even quicker sweep with default parameters
# sweep_id = quick_sweep(
#     dataset=dataset,
#     project_name=project_name,
#     hidden_channels=[32, 64],
#     num_layers=[2, 3],
#     dropout=[0.15, 0.25],
#     lr=[0.001, 0.005],
#     weight_decay=[0, 1e-4],
#     epochs=50  # Fewer epochs for testing
# )

## 4. Analyze Sweep Results

In [ ]:
# Replace with your actual sweep ID
# sweep_id = "your-sweep-id-here"
# results = analyze_sweep_results(project_name, sweep_id)

In [ ]:
# Display best configuration
# if results['best_config']:
#     print("Best Configuration Found:")
#     print(f"Validation Accuracy: {results['best_config']['best_val_accuracy']:.4f}")
#     print("\nHyperparameters:")
#     for param, value in results['best_config']['config'].items():
#         if param in param_ranges:
#             print(f"  {param}: {value}")

In [ ]:
# Show top N configurations
# N = 10
# print(f"\nTop {N} Configurations:")
# for i, config in enumerate(results['all_results'][:N]):
#     print(f"\n{i+1}. Validation Accuracy: {config['best_val_accuracy']:.4f}")
#     print("   Config:", {k: v for k, v in config['config'].items() if k in param_ranges})

## 5. Visualize Results

In [ ]:
# Function to visualize hyperparameter importance
def plot_hyperparameter_importance(results, param_name):
    """Plot validation accuracy distribution for different values of a hyperparameter."""
    if not results['all_results']:
        print("No results to plot")
        return
    
    # Extract data
    param_values = []
    accuracies = []
    
    for run in results['all_results']:
        if param_name in run['config']:
            param_values.append(run['config'][param_name])
            accuracies.append(run['best_val_accuracy'])
    
    # Create DataFrame
    df = pd.DataFrame({
        param_name: param_values,
        'validation_accuracy': accuracies
    })
    
    # Plot
    plt.figure(figsize=(10, 6))
    sns.boxplot(x=param_name, y='validation_accuracy', data=df)
    plt.title(f'Validation Accuracy vs {param_name}')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# Plot for each hyperparameter
# for param in param_ranges.keys():
#     plot_hyperparameter_importance(results, param)

In [ ]:
# Create a heatmap for two hyperparameters
def plot_2d_heatmap(results, param1, param2):
    """Create a heatmap showing validation accuracy for two hyperparameters."""
    if not results['all_results']:
        print("No results to plot")
        return
    
    # Extract data
    data = {}
    for run in results['all_results']:
        if param1 in run['config'] and param2 in run['config']:
            key = (run['config'][param1], run['config'][param2])
            if key not in data or run['best_val_accuracy'] > data[key]:
                data[key] = run['best_val_accuracy']
    
    # Create matrix
    param1_values = sorted(set(k[0] for k in data.keys()))
    param2_values = sorted(set(k[1] for k in data.keys()))
    
    matrix = np.zeros((len(param2_values), len(param1_values)))
    for i, p2 in enumerate(param2_values):
        for j, p1 in enumerate(param1_values):
            matrix[i, j] = data.get((p1, p2), 0)
    
    # Plot
    plt.figure(figsize=(10, 8))
    sns.heatmap(matrix, 
                xticklabels=param1_values, 
                yticklabels=param2_values,
                annot=True, 
                fmt='.3f', 
                cmap='viridis')
    plt.xlabel(param1)
    plt.ylabel(param2)
    plt.title(f'Validation Accuracy: {param1} vs {param2}')
    plt.tight_layout()
    plt.show()

In [ ]:
# Plot heatmaps for interesting parameter pairs
# plot_2d_heatmap(results, 'hidden_channels', 'num_layers')
# plot_2d_heatmap(results, 'lr', 'weight_decay')
# plot_2d_heatmap(results, 'hidden_channels', 'dropout')

## 6. Save Results

In [ ]:
# Save results to file
# import json
# with open(f'sweep_results_{sweep_id}.json', 'w') as f:
#     json.dump(results, f, indent=2)
# print(f"Results saved to sweep_results_{sweep_id}.json")

In [ ]:
# Create a summary DataFrame
# if results['all_results']:
#     summary_data = []
#     for run in results['all_results']:
#         row = {
#             'val_accuracy': run['best_val_accuracy'],
#             'train_accuracy': run['final_train_accuracy'],
#             'best_epoch': run['best_epoch']
#         }
#         # Add hyperparameters
#         for param in param_ranges.keys():
#             if param in run['config']:
#                 row[param] = run['config'][param]
#         summary_data.append(row)
#     
#     summary_df = pd.DataFrame(summary_data)
#     summary_df.to_csv(f'sweep_summary_{sweep_id}.csv', index=False)
#     print("Summary saved to CSV")
#     print(summary_df.head())

## 7. Train Final Model with Best Hyperparameters

In [ ]:
# Extract best hyperparameters
# if results['best_config']:
#     best_params = results['best_config']['config']
#     
#     # Create configuration for final training
#     final_config = SimpleNamespace(
#         model_name='gin',
#         hidden_channels=best_params['hidden_channels'],
#         num_layers=best_params['num_layers'],
#         dropout=best_params['dropout'],
#         lr=best_params['lr'],
#         weight_decay=best_params['weight_decay'],
#         epochs=150,  # Train longer for final model
#         batch_size=32,
#         scheduler_type='onecycle',
#         use_wandb=True,
#         project='gnn-planar-graphs-final',
#         experiment_name='best_model_from_sweep',
#         in_channels=dataset[0].x.shape[1]
#     )
#     
#     # Train final model
#     from training_utils import train
#     final_results = train(final_config, dataset)
#     
#     print(f"Final model validation accuracy: {final_results['best_val_acc']:.4f}")
#     
#     # Save the final model
#     torch.save(final_results['model_state'], 'best_model_from_sweep.pt')